In [2]:
import pandas as pd
import os
from PIL import Image, ImageOps
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from torchvision import models
import torch.optim as optim
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import train_test_split
from scripts.model import DRDetection
from scripts.loss import FocalLoss
from tqdm import tqdm

/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/python-3.9.9-jh/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
script_name = os.path.splitext(os.path.basename(__file__))[0]

In[2]:

In [3]:
MEAN = [0.41661593, 0.29097975, 0.20843531]
STD = [0.26398131, 0.19219237, 0.15810781]

# Custom class for dataset loading

In[3]:

In [4]:
class CustomImageDataset(Dataset):
    def __init__(self, file_paths, labels=None, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform
        # self.inverted = inverted  # New column for inversion status
    def __len__(self):
        return len(self.file_paths)
    def __getitem__(self, idx):
        img_path = self.file_paths[idx]
        image = Image.open(img_path + '.jpeg').convert("RGB")
        
        # # Check and handle inversion
        # if self.inverted is not None and self.inverted[idx] == 1:
        #     image = ImageOps.flip(image)  # Un-invert the image
        if self.transform:
            image = self.transform(image)
        if self.labels is not None:  # For training and validation
            label = self.labels[idx]
            return image, label
        else:  # For testing
            return image

In[4]:

In [5]:
train_transform = transforms.Compose([
            # transforms.Resize((380, 380)),
            # transforms.RandomHorizontalFlip(p=0.5),                    # Random horizontal flip
            # transforms.ColorJitter(brightness=0.2, contrast=0.2),      # Random brightness/contrast
            transforms.ToTensor(),                                    # Convert to Tensor
            transforms.Normalize(mean=MEAN,         
                                 std=STD),
        ])

In [6]:
val_test_transform = transforms.Compose([
            # transforms.Resize((380, 380)),
            transforms.ToTensor(),
            transforms.Normalize(mean=MEAN,
                                 std=STD),
        ])

In[5]:

Load CSV

In [7]:
csv_path = "trainLabels.csv"
data = pd.read_csv(csv_path)

In [8]:
class_weights = dict(len(data)/data['level'].value_counts())
class_weights = torch.tensor([class_weights[i] for i in sorted(class_weights.keys())], dtype=torch.float)

Add full paths to image files

In [9]:
train_folder = "train_prep512/"
data['image_path'] = data['image'].apply(lambda x: os.path.join(train_folder, x))

inverted = data['inverted'].values

Stratified split for training and validation

In [10]:
train_paths, val_paths, train_labels, val_labels = train_test_split(
    data['image_path'].values,
    data['level'].values,
    # inverted,
    test_size=0.1,  # 10% for validation
    stratify=data['level'].values,
    random_state=42
)

# Loading the dataset

In[6]:

Training and Validation datasets

In [11]:
train_dataset = CustomImageDataset(train_paths, train_labels, transform=train_transform)
val_dataset = CustomImageDataset(val_paths, val_labels, transform=val_test_transform)

Data loaders

In [12]:
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Model building

In[15]:

In [13]:
def train_model_with_validation(device, model, train_loader, val_loader, optimizer, criterion, scheduler=None, num_epochs=10):
    model.train()
    training_loss, validation_loss = [], []
    training_acc, validation_acc = [], []
    kappa_score = []
    for epoch in range(num_epochs):
        # Training phase
        running_loss = 0.0
        correct = 0
        total = 0
        for images, labels in tqdm(train_loader):
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images) #inceptionv3 returns two outputs
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        train_accuracy = 100 * correct / total
        train_loss = running_loss / len(train_loader)

        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        y_true = []
        y_pred = []
        with torch.no_grad():
            for images, labels in tqdm(val_loader):
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
                # Collect predictions and true labels for Kappa calculation
                y_true.extend(labels.cpu().numpy())
                y_pred.extend(predicted.cpu().numpy())
        val_accuracy = 100 * val_correct / val_total
        val_loss /= len(val_loader)

        # Quadratic Weighted Kappa Score
        kappa = cohen_kappa_score(y_true, y_pred, weights="quadratic")
        training_loss.append(train_loss)
        validation_loss.append(val_loss)
        training_acc.append(train_accuracy)
        validation_acc.append(val_accuracy)
        kappa_score.append(kappa)
        if scheduler:
            scheduler.step()

        # Print epoch metrics
        print(f"Epoch [{epoch+1}/{num_epochs}]")
        print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%")
        print(f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%, Kappa Score: {kappa:.4f}\n")
        model.train()  # Switch back to training mode
        
    return training_loss, validation_loss, training_acc, validation_acc, kappa_score

In [26]:
def init_model():
    # model = models.efficientnet_b4(weights=models.EfficientNet_B4_Weights.DEFAULT)
    # num_features = model.classifier[1].in_features
    # model.classifier[1] = nn.Linear(num_features, 5)
    model = DRDetection(dropout_prob=0.5, leakiness=0.05)
    
    return model

In[18]:

In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = init_model().to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))  # You can pass class weights here if needed

In [28]:
# criterion = FocalLoss(gamma=3).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
# optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.9, weight_decay=1e-4, nesterov=True)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
scheduler = None

In[19]:

In [29]:
epochs=10
training_loss, validation_loss, training_acc, validation_acc, kappa_score = train_model_with_validation(device, model, train_loader, val_loader, optimizer, criterion, scheduler, epochs)
torch.save(model.state_dict(), f"model/{script_name}_model.pth")

100%|██████████| 110/110 [00:07<00:00, 14.89it/s]


Epoch [1/10]
Train Loss: 38.8544, Train Accuracy: 22.09%
Val Loss: 3.7702, Val Accuracy: 14.80%, Kappa Score: 0.0020



100%|██████████| 110/110 [00:06<00:00, 15.94it/s]


Epoch [2/10]
Train Loss: 3.2894, Train Accuracy: 22.99%
Val Loss: 1.7741, Val Accuracy: 3.16%, Kappa Score: 0.0009



100%|██████████| 110/110 [00:07<00:00, 14.41it/s]


...

Epoch [5/10]
Train Loss: 1.7944, Train Accuracy: 28.12%
Val Loss: 1.6759, Val Accuracy: 15.49%, Kappa Score: -0.0034



100%|██████████| 110/110 [00:07<00:00, 15.03it/s]


Epoch [6/10]
Train Loss: 1.7207, Train Accuracy: 32.93%
Val Loss: 1.6429, Val Accuracy: 6.97%, Kappa Score: 0.0000



  3%|▎         | 25/988 [00:04<03:02,  5.28it/s]


KeyboardInterrupt: 

In[21]:

Plot Loss

In [ ]:
plt.figure(figsize=(12, 6))

In [ ]:
plt.subplot(1, 2, 1)  # 1 row, 2 columns, 1st subplot
plt.plot(training_loss, label='Training Loss', marker='o')
plt.plot(validation_loss, label='Validation Loss', marker='o')
plt.title('Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid()

Plot Accuracy

In [ ]:
plt.subplot(1, 2, 2)  # 1 row, 2 columns, 2nd subplot
plt.plot(training_acc, label='Training Accuracy', marker='o')
plt.plot(validation_acc, label='Validation Accuracy', marker='o')
plt.title('Accuracy over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid()

In [ ]:
plt.tight_layout()
plt.savefig(f'images/{script_name}_test_val.png')

Plot Kappa Score

In [ ]:
plt.figure(figsize=(6, 4))
plt.plot(kappa_score, label='Kappa Score', marker='o', color='purple')
plt.title('Kappa Score over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Kappa Score')
plt.legend()
plt.grid()
plt.savefig(f'images/{script_name}_kappa.png')

In [ ]:
test_folder = "test_prep512/"
test_files = os.listdir(test_folder)

In [ ]:
results = []

In [ ]:
model.eval()

In [ ]:
for f in test_files:
    img = Image.open(os.path.join(test_folder, f)).convert('RGB')
    with torch.no_grad():
        outputs = model(val_test_transform(img).unsqueeze(0).to(device))
        _, pred = torch.max(outputs, 1)  # Get the class index with the highest score
        results.append([os.path.splitext(f)[0], pred.item()])

Save to a single CSV

In [ ]:
df = pd.DataFrame(results, columns=["image", "level"])
df.to_csv(f"output/{script_name}_predictions.csv", index=False)
print("Predictions saved")